# 5. 셀레니움 활용 웹브라우저 자동화

In [1]:
## freeze 할때는 이 코드를 써서 안쓰는 패키지를 잘못 freeze 하지 않게 조심하기

In [2]:
# #설치 되어있는 패키지만을 freeze
# import sys, pkg_resources
# # 현재 메모리에 로드된 모듈 이름 목록을 리스트로 저장
# # sys.modules는 실행 중에 동적으로 import된 모듈까지 포함
# mods = list(sys.modules)
# # import된 패키지들의 (이름, 버전) 튜플을 저장할 리스트
# pkgs = []
# # 설치된 모든 패키지 정보 가져오기
# dists = list(pkg_resources.working_set)

# # 설치된 패키지 중 현재 import된 것만 골라서 pkgs에 추가
# for i in range(0, len(dists)):
#     name = dists[i].project_name
#     key = dists[i].key
#     if name in mods or key in mods:
#         pkgs.append((name, dists[i].version))

# # requirements.txt 파일로 저장
# with open('requirements.txt', 'w') as f:
#     for i in range(0, len(pkgs)):
#         f.write(f"{pkgs[i][0]}=={pkgs[i][1]}\n")

# for i in range(0, len(dists)):
#     name = dists[i].project_name
#     key = dists[i].key
#     if name in mods or key in mods:
#         pkgs.append((name, dists[i].version))

# # requirements.txt 파일로 저장
# with open('requirements.txt', 'w') as f:
#     for i in range(0, len(pkgs)):
#         f.write(f"{pkgs[i][0]}=={pkgs[i][1]}\n")

# print("requirements.txt 생성 완료")

In [ ]:
# !pip install -U PyMySQL==1.1.1
# !pip install -U numpy==2.0.2
# !pip install -U pandas==2.2.2
# !pip install -U requests==2.32.3
# !pip install -U selenium==4.31.0
# !pip install -U sqlalchemy==2.0.40
# !pip install -U webdriver_manager==4.0.2
# !pip install -U bs4==0.0.2

In [4]:
# # 2. Chrome 브라우저 설치
# !apt-get update  # 패키지 목록 업데이트
# !apt-get install -y wget unzip  # wget과 unzip 패키지 설치
# !wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | apt-key add -  # Google의 리눅스 서명 키 추가
# !sh -c 'echo "deb [arch=amd64] https://dl.google.com/linux/chrome/deb/ stable main" >> /etc/apt/sources.list.d/google-chrome.list'  # Chrome 브라우저를 다운로드할 수 있도록 저장소 추가
# !apt-get update  # 패키지 목록 다시 업데이트
# !apt-get install -y google-chrome-stable  # 최신 안정 버전의 Chrome 브라우저 설치
# !apt-get install -y fonts-nanum # 한글폰트 설치

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [73.0 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,694 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages 

In [5]:
# pip freeze >> requirements.txt

### 라이브러리 선언

In [6]:
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine, inspect
# Selenium과 webdriver-manager를 사용한 Chrome 브라우저 실행 코드
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import bs4

### 크롬브라우저 옵션 설정

In [7]:
# WebDriver 설치 및 브라우저 실행 설정
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # ★★★헤드리스 모드로 실행 (브라우저 창을 표시하지 않음)
options.add_argument("window-size=1920x1080") # 헤드리스 모드로 실행 (명시적으로 표기)
options.add_argument("--no-sandbox")  # 샌드박스 모드 비활성화 (가상 환경에서 안정적인 실행을 위해)
options.add_argument("--disable-dev-shm-usage")  # /dev/shm 사용 비활성화 (메모리 부족 방지)
options.add_argument("--disable-gpu")  # GPU 가속 비활성화 (필요시 사용)
options.add_argument("start-maximized")  # 브라우저 창 최대화
options.add_argument("disable-infobars")  # 정보 표시줄 비활성화
options.add_argument("--disable-extensions")  # 브라우저 확장 프로그램 비활성화
options.add_argument('--ignore-certificate-errors') # 통신 키 제거

# user-agent 값 설정
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
options.add_argument(f"user-agent={user_agent}")

# WebDriver로 Chrome 브라우저 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
# 웹페이지 파싱 될때까지 최대 3초 기다림
driver.implicitly_wait(3)

### 1. 스파크 퀴즈 정보 수집

In [8]:
try:
    targetUrl = "https://sparkkorea.com/"
    driver.get( targetUrl )
    driver.current_url
    driver.save_screenshot("./here.jpg")
    ### 1. 원하는 액션대상 요소 탐색
    quizXpath = '//*[@id="menu-item-382"]/a'
    quizMenuEle = driver.find_element(By.XPATH, quizXpath)
    ### 2. 액션대상 요소 클릭
    quizMenuEle.click()

    ### 3. 페이지 소스 불러오기
    htmlObj = driver.page_source
    bsObj = bs4.BeautifulSoup(htmlObj, "html.parser")

    ### 4. 퀴즈정보 수집
    sparkQuizDiv = bsObj.find(name = "div", attrs={"class":"class_spark_quiz"})
    aLinks = sparkQuizDiv.find_all(name="a")
    linkList = []
    nameList = []
    for i in range(0, len(aLinks)):
        eachAtag = aLinks[i]
        nameList.append( eachAtag.text )
        linkList.append( eachAtag.attrs["href"] )
    quizDf = pd.DataFrame( zip (nameList, linkList), columns = ["링크명","링크주소"] )
    quizDf
except Exception as e:
    print(e)

/usr/local/lib/python3.11/dist-packages/selenium/webdriver/remote/webdriver.py:994: UserWarning: name used for saved screenshot does not match file type. It should end with a `.png` extension
  return self.get_screenshot_as_file(filename)


### 2. 데이터베이스 저장

In [9]:
now = datetime.now()
time_key = now.strftime("%Y_%m_%d_%H_%M_%S")
quizDf["TIME_KEY"] = time_key

In [10]:
quizDf.columns = ["LINK_NAME","LINK_ADDR","TIME_KEY"]

In [11]:
# MySQL 데이터베이스 연결 정보 설정 (ngrok)
### 5. DB에 저장하기
### - IP: my8003.gabiadb.com
### - 포트: 3306
### - DB명: hkcodedb
### - 계정: hkcode / fintech1308!
user = 'hkcode'
password = 'fintech1308!'
host = 'my8003.gabiadb.com'
port = '3306'
database = 'hkcodedb'

# SQLAlchemy 엔진 생성
engineMy = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}')
# 데이터프레임을 MySQL 데이터베이스의 테이블에 저장
# indata.to_sql(name="ngrok_test", con=engineMy, if_exists='replace', index=False)

In [12]:
try:
    tableName = "service_hkcode_0407"
    quizDf.to_sql(name= tableName,index=False, if_exists="append", con=engineMy)
    print("DB이관 완료")
except Exception as e:
    print(e)

DB이관 완료


In [26]:
import sys, pkg_resources

# 현재 메모리에 로드된 모듈 이름 목록을 리스트로 저장
# sys.modules는 실행 중에 동적으로 import된 모듈까지 포함
# 예: pandas 하나만 import했더라도 내부적으로 numpy, dateutil 등 여러 모듈을 import해서 메모리 올림
mods = list(sys.modules)

# import된 패키지들의 (이름, 버전) 튜플을 저장할 리스트
pkgs = []

# 설치된 모든 패키지 정보 가져오기
dists = list(pkg_resources.working_set)

# 설치된 패키지 중 현재 import된 것만 골라서 pkgs에 추가
for i in range(0, len(dists)):
    name = dists[i].project_name
    key = dists[i].key
    if name in mods or key in mods:
        pkgs.append((name, dists[i].version))

# requirements.txt 파일로 저장
with open('requirements.txt', 'w') as f:
    for i in range(0, len(pkgs)):
        f.write(f"{pkgs[i][0]}=={pkgs[i][1]}\n")

print("requirements.txt 생성 완료")

requirements.txt 생성 완료
